In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import geopandas as gpd
import fiona
import os

os.environ["PROJ_LIB"] = r"C:\\Users\\SQwan\\miniconda3\\Library\\share"
import pickle
import googlemaps
import networkx as nx
import itertools
import csv
import glob
from collections import Counter
from datetime import datetime
from pprint import pprint
from mpl_toolkits.basemap import Basemap
from python_tsp.exact import solve_tsp_dynamic_programming
from python_tsp.distances import great_circle_distance_matrix
from shapely.geometry import Point, shape, Polygon
from shapely.ops import unary_union

import shapely.speedups

shapely.speedups.enable()

if not os.path.exists(r"Data"):
    os.makedirs(r"Data")

if not os.path.exists(r"Data\temp"):
    os.makedirs(r"Data\temp")

## Blue, red, and yellow route stations

In [2]:
blue_station = [Point(42.116034, -86.452483),
                Point(42.115074, -86.465348),
                Point(42.110256, -86.463736),
                Point(42.110254, -86.457768),
                Point(42.101634, -86.448961),
                Point(42.101646, -86.441657),
                Point(42.102544, -86.436052),
                Point(42.088709, -86.437108),
                Point(42.084822, -86.437156),
                Point(42.080667, -86.434759),
                Point(42.085583, -86.433805),
                Point(42.085637, -86.424232),
                Point(42.082548, -86.421979),
                Point(42.082242, -86.418849),
                Point(42.077808, -86.424668),
                Point(42.102544, -86.436052),
                Point(42.107206, -86.446024),
                Point(42.109733, -86.447242),
                Point(42.116034, -86.452483),
                ]
blue_station = [Point(p.y, p.x) for p in blue_station]

red_station = [Point(42.101646,	-86.441657),
               Point(42.101634,	-86.448961),
               Point(42.116034,	-86.452483),
               Point(42.115074,	-86.465348),
               Point(42.111264,	-86.481872),
               Point(42.088810,	-86.478394),
               Point(42.084126,	-86.486379),
               Point(42.079074,	-86.493490),
               Point(42.033439,	-86.513542),
               Point(42.026502,	-86.516012),
               Point(42.086425,	-86.440537),
               Point(42.101646,	-86.441657),
               ]
red_station = [Point(p.y, p.x) for p in red_station]

yellow_station = [Point(42.118494913335645, -86.45082973186932),
                  Point(42.13082775201815, -86.4538851865351),
                  Point(42.13268958444188, -86.45128880811971),
                  Point(42.124573800847095, -86.4460383743168),
                  Point(42.121903066372475, -86.4390957589761),
                  Point(42.116026992072754, -86.4296080933503),
                  Point(42.11587877166418, -86.43641202669362),
                  Point(42.112791181420455, -86.4407060644722),
                  Point(42.10241413329736, -86.43602474092258),
                  Point(42.10241413, -86.43602474),
                  Point(42.11279118, -86.44070606),
                  Point(42.11587877, -86.43641203),
                  Point(42.11602699, -86.42960809),
                  Point(42.12190307, -86.43909576),
                  Point(42.1245738,	-86.44603837),
                  Point(42.13268958, -86.45128881),
                  Point(42.13082775	, -86.45388519),
                  Point(42.11849491, -86.45082973),
                  ]
yellow_station = [Point(p.y, p.x) for p in yellow_station]

In [3]:
def locate_point(point, geoseries):
    """
    Checker function which determines if a coordinate is within 
    the geoseries and return the zone id where the point is 
    located
    """
    idx_list = geoseries.index[geoseries.geometry.contains(point)].tolist()
    return idx_list[0] if len(idx_list) >= 1 else np.nan

In [4]:
gdf = gpd.read_file(r"Data\shapefile\zone_id_agg.shp")

In [5]:
blue_station_id = [gdf.loc[locate_point(p, gdf), "zone_id"] for p in blue_station]
blue_station_id = [
    v
    for i, v in enumerate(blue_station_id)
    if i < len(blue_station_id) - 1
    if v != blue_station_id[i + 1]
]

blue_station_id += [gdf.loc[locate_point(blue_station[-1], gdf), "zone_id"]]

red_station_id = [gdf.loc[locate_point(p, gdf), "zone_id"] for p in red_station]
red_station_id = [
    v
    for i, v in enumerate(red_station_id)
    if i < len(red_station_id) - 1
    if v != red_station_id[i + 1]
]

red_station_id += [gdf.loc[locate_point(red_station[-1], gdf), "zone_id"]]

yellow_station_id = [gdf.loc[locate_point(p, gdf), "zone_id"] for p in yellow_station]
yellow_station_id = [
    v
    for i, v in enumerate(yellow_station_id)
    if i < len(yellow_station_id) - 1
    if v != yellow_station_id[i + 1]
]

yellow_station_id += [gdf.loc[locate_point(yellow_station[-1], gdf), "zone_id"]]

In [6]:
# Load neighbor nodes information
ctr_info = pickle.load(open(r"Data\temp\Station_agg.p", "rb"))
# graph with edge cost as shortest travel time
G_t = pickle.load(open(r"Data\temp\G_t_agg.p", "rb"))

s_blue = list()
s_blue.append(blue_station_id[0])
for previous, current in zip(blue_station_id, blue_station_id[1:]):
    if current in ctr_info[previous]["neighbours"]:
        s_blue.append(current)
    else:
        sp = nx.shortest_path(G_t, source=previous, target=current, weight="weight")
        s_blue += sp[1:]

s_red = list()
s_red.append(red_station_id[0])
for previous, current in zip(red_station_id, red_station_id[1:]):
    if current in ctr_info[previous]["neighbours"]:
        s_red.append(current)
    else:
        sp = nx.shortest_path(G_t, source=previous, target=current, weight="weight")
        s_red += sp[1:]

s_yellow = list()
s_yellow.append(yellow_station_id[0])
for previous, current in zip(yellow_station_id, yellow_station_id[1:]):
    if current in ctr_info[previous]["neighbours"]:
        s_yellow.append(current)
    else:
        sp = nx.shortest_path(G_t, source=previous, target=current, weight="weight")
        s_yellow += sp[1:]

### Get bus schedule time based on tau matrix

In [7]:
# graph with edge cost as shortest travel time
G_t = pickle.load(open(r"Data\temp\G_t_agg.p", "rb"))

S = 39  # number of zones
DELTA_t = 1  # x min
tau = np.zeros((S, S))
tau2 = np.zeros((S, S))
# round travel time to integer
for _, _, d in G_t.edges(data=True):
    d["weight"] = np.rint(d["weight"])

for i in range(S):
    for j in range(S):
        if i == j:
            tau[i, j] = 0
            tau2[i, j] = 1
        else:
            tau[i, j] = (
                nx.shortest_path_length(G_t, source=i, target=j, weight="weight")
                // DELTA_t
            )
            tau2[i, j] = tau[i, j]

In [8]:
sch_blue_1 = np.cumsum(
    [0]
    + [
        tau[s_blue[i], s_blue[i + 1]]
        if s_blue[i] != s_blue[i + 1]
        else 1
        for i in range(len(s_blue) - 1)
    ]
)

# Rotate the blue bus station list with 30 mins
idx = np.where(sch_blue_1 >= 30)[0][0]
s_blue_2 = s_blue[-idx:] + s_blue[:-idx]

sch_blue_2 = np.cumsum(
    [0]
    + [
        tau[s_blue_2[i], s_blue_2[i + 1]]
        if s_blue_2[i] != s_blue_2[i + 1]
        else 1
        for i in range(len(s_blue_2) - 1)
    ]
)


sch_red_1 = np.cumsum(
    [0]
    + [
        tau[s_red[i], s_red[i + 1]]
        if s_red[i] != s_red[i + 1]
        else 1
        for i in range(len(s_red) - 1)
    ]
)

sch_yellow_1 = np.cumsum(
    [0]
    + [
        tau[s_yellow[i], s_yellow[i + 1]]
        if s_yellow[i] != s_yellow[i + 1]
        else 1
        for i in range(len(s_yellow) - 1)
    ]
)

In [10]:
blue_fr_1 = {"t": sch_blue_1, "s": s_blue * 1}
blue_fr_2 = {"t": sch_blue_2, "s": s_blue_2 * 1}

df_blue_fr_1 = pd.DataFrame.from_dict(blue_fr_1)
df_blue_fr_2 = pd.DataFrame.from_dict(blue_fr_2)

red_fr_1 = {"t": sch_red_1, "s": s_red * 1}

df_red_fr_1 = pd.DataFrame.from_dict(red_fr_1)

yellow_fr_1 = {"t": sch_yellow_1, "s": s_yellow * 1}

df_yellow_fr_1 = pd.DataFrame.from_dict(yellow_fr_1)

In [11]:
df_blue_fr_1.to_csv(r"many2many_data\blue_fr_1_agg.csv", index=False)
df_blue_fr_2.to_csv(r"many2many_data\blue_fr_2_agg.csv", index=False)
df_red_fr_1.to_csv(r"many2many_data\red_fr_1_agg.csv", index=False)
df_yellow_fr_1.to_csv(r"many2many_data\yellow_fr_1_agg.csv", index=False)

# Zone Disaggregation

In [2]:
agg_2_disagg_id = pickle.load(open(r"Data\agg_2_disagg_id.p", "rb"))
disagg_2_agg_id = pickle.load(open(r"Data\disagg_2_agg_id.p", "rb"))

In [3]:
route_file_path = r"E:\Codes\BH_data_preprocess_and_DP\test_data\result\3_28_2021\\"

route_dict = {}

for i, f in enumerate(glob.glob(route_file_path + "route_*_agg.csv")):
    route_dict[i] = pd.read_csv(f, encoding="utf-8")
    
N_route = len(route_dict) # number of route

pth_dict_agg = {}
for k, route in route_dict.items():
    pth_dict_agg[k] = list(set((route["s1"])))

In [4]:
pth_dict_temp = {}
for k, pth in pth_dict_agg.items():
    pth_dict_temp[k] = [agg_2_disagg_id[s] for s in pth]

pth_dict = {}    
for k, pth in pth_dict_temp.items():
    pth_dict[k] = list()
    for lst in pth:
        pth_dict[k] += lst

# Solve the Open TSP Problem
After solving the aggregated many-to-many problem, solve the disaggregated sub-problem as an open TSP problem.

In [5]:
ctr_info = pickle.load(open(r"Data\temp\Station.p", "rb"))
c_dict = {
    k: [ctr_info[zone]["lat_lon"] for zone in zone_lst]
    for k, zone_lst in pth_dict.items()
} # centroid coordinates of zones

In [70]:
# graph with edge cost as shortest travel time
G_t = pickle.load(open(r"Data\temp\G_t.p", "rb"))

S = 69  # number of zones
DELTA_t = 1  # x min
tau = np.zeros((S, S))
tau2 = np.zeros((S, S))
# round travel time to integer
for _, _, d in G_t.edges(data=True):
    d["weight"] = np.rint(d["weight"])

for i in range(S):
    for j in range(S):
        if i == j:
            tau[i, j] = 0
            tau2[i, j] = 1
        else:
            tau[i, j] = (
                nx.shortest_path_length(G_t, source=i, target=j, weight="weight")
                // DELTA_t
            )
            tau2[i, j] = tau[i, j]

In [73]:
distance_matrix = np.zeros((len(c_dict[4]), len(c_dict[4])))
for i in range(len(c_dict[4])):
    for j in range(len(c_dict[4])):
        distance_matrix[i, j] = tau[pth_dict[4][i], pth_dict[4][j]]

In [75]:
# distance_matrix = great_circle_distance_matrix(c_dict[4]) # only the 5-th route is optimized
permutation, distance = solve_tsp_dynamic_programming(distance_matrix)

[0, 7, 5, 6, 8, 17, 18, 16, 14, 15, 13, 12, 11, 4, 3, 1, 2, 10, 9]

## Check if all pair of successive zones are neighbors

In [76]:
route_id = [pth_dict[4][i] for i in permutation]
# Load neighbor nodes information
ctr_info = pickle.load(open(r"Data\temp\Station.p", "rb"))
# graph with edge cost as shortest travel time
G_t = pickle.load(open(r"Data\temp\G_t.p", "rb"))

s = list()
s.append(route_id[0])
for previous, current in zip(route_id, route_id[1:]):
    if current in ctr_info[previous]["neighbours"]:
        s.append(current)
    else:
        sp = nx.shortest_path(G_t, source=previous, target=current, weight="weight")
        s += sp[1:]

# make sure the bus back to the depot
s.append(s[0])

## Get bus schedule time based on tau matrix

In [77]:
# graph with edge cost as shortest travel time
G_t = pickle.load(open(r"Data\temp\G_t.p", "rb"))

S = 69  # number of zones
DELTA_t = 1  # x min
tau = np.zeros((S, S))
tau2 = np.zeros((S, S))
# round travel time to integer
for _, _, d in G_t.edges(data=True):
    d["weight"] = np.rint(d["weight"])

for i in range(S):
    for j in range(S):
        if i == j:
            tau[i, j] = 0
            tau2[i, j] = 1
        else:
            tau[i, j] = (
                nx.shortest_path_length(G_t, source=i, target=j, weight="weight")
                // DELTA_t
            )
            tau2[i, j] = tau[i, j]

In [78]:
sch = np.cumsum(
    [0] + [tau[s[i], s[i + 1]] if s[i] != s[i + 1] else 1 for i in range(len(s) - 1)]
)
fr = {"t": sch, "s": s * 1}

df_fr = pd.DataFrame.from_dict(fr)
df_fr.to_csv(r"many2many_output\fr_1.csv", index=False)

In [79]:
df_sch = pd.DataFrame()
df_sch["t1"], df_sch["t2"] = fr["t"][:-1], fr["t"][1:]
df_sch["s1"], df_sch["s2"] = fr["s"][:-1], fr["s"][1:]
df_sch.to_csv(r"many2many_output\sch_1.csv", index=False)

### Open TSP Problem模板

In [ ]:
sources = np.array([],)  # centroid coordinates of zones
distance_matrix = great_circle_distance_matrix(sources)
# set all elements of the first column of the distance matrix to zero
distance_matrix[:, 0] = 0
permutation, distance = solve_tsp_dynamic_programming(distance_matrix)